# Imports

In [ ]:
##Imports Libraries using in my project
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
##Read Books.Csv and show them
booksDataset = pd.read_csv('dataset/Books.csv')
booksDataset.head()

## Clearing Dataset

In [ ]:
##Counts Null Values
booksDataset.isnull().sum()

In [ ]:
booksDataset=booksDataset.dropna() ##Remove all null values in dataset
booksDataset.isnull().sum()##Display dataset after removing null values

In [ ]:
##Display formate type for each columns in dataset
booksDataset.info()

In [ ]:
##Describe columns in dataset
booksDataset.describe()

In [ ]:
##Sorting average rating in descending order
booksDataset = booksDataset.sort_values('average_rating')
##Reset index from 0 in average rating
booksDataset = booksDataset.reset_index()
booksDataset

In [ ]:
##Casting type of average_Rating columns to float type
booksDataset.average_rating = booksDataset.average_rating.astype(float)

In [ ]:
##Display First 50 Heading in Books Datasets
booksDataset.tail(50)

In [ ]:
##Sorting average rating in descending order
booksDataset = booksDataset.sort_values('num_pages')
booksDataset

# Data Vaisualization Graphs

In [ ]:
plt.figure(figsize=(20,20))
ax = sns.relplot(data=booksDataset, x="average_rating", y="ratings_count",sizes=(400,200), height=7)
plt.title('The Relation Between Rating Counts & Average Rating')
ax.set_axis_labels("average_rating", "ratings_count")

In [ ]:
##Top 10 Books
top_ten_books = booksDataset[booksDataset['ratings_count']>1000000]
top_ten_books = top_ten_books.sort_values(by='ratings_count', ascending=False).head(10)

top_ten_books_rating = top_ten_books.sort_values(by='average_rating', ascending=False)

plt.style.use('seaborn-whitegrid')
plt.figure(figsize=(8,8))
sns.barplot(x="average_rating", y="title", data = top_ten_books_rating, palette='inferno')

In [ ]:
##Display formate type for each columns in dataset
booksDataset.info()

In [ ]:
##Casting type of num_pages columns to float type
booksDataset.num_pages = booksDataset.num_pages.astype(float)
booksDataset.info()

In [ ]:
##Top 50 Books
top_fifty_books = booksDataset[booksDataset['ratings_count']>10000]
top_fifty_books = top_ten_books.sort_values(by='ratings_count', ascending=False).head(50)
top_fifty_books_rating = top_ten_books.sort_values(by='average_rating', ascending=False)
top_fifty = top_fifty_books_rating.title
top_fifty

# Data Prepration

In [ ]:
##Copy dataset in new variablea
booksDataset_2 = booksDataset.copy()

In [ ]:
##Adjst average ratings
booksDataset_2.loc[(booksDataset_2['average_rating']>=0)& (booksDataset_2['average_rating']<=1),'rating between']="between 0 and 1"
booksDataset_2.loc[(booksDataset_2['average_rating']>1) & (booksDataset_2['average_rating']<=2),'rating between']="between 1 and 2"
booksDataset_2.loc[(booksDataset_2['average_rating']>2) & (booksDataset_2['average_rating']<=3),'rating between']="between 2 and 3"
booksDataset_2.loc[(booksDataset_2['average_rating']>3) & (booksDataset_2['average_rating']<=4),'rating between']="between 3 and 4"
booksDataset_2.loc[(booksDataset_2['average_rating']>4) & (booksDataset_2['average_rating']<=5),'rating between']="between 4 and 5"

In [ ]:
##Sorting average rating in descending order
booksDataset_2.sort_values('average_rating')

In [ ]:
booksDataset_rating = pd.get_dummies(booksDataset_2['rating between'])
booksDataset_categories = pd.get_dummies(booksDataset_2['categories'])

In [ ]:
##Display ratings between
booksDataset_rating

In [ ]:
##Display categories
booksDataset_categories

In [ ]:
feature = pd.concat([booksDataset_rating, booksDataset_categories, booksDataset_2['average_rating'], booksDataset_2['categories']], axis=1)

In [ ]:
feature.drop(['categories'], axis='columns', inplace=True)
feature

In [ ]:
feature.info()

# Building Book Recommendation System

In [ ]:
from sklearn import neighbors
from sklearn.model_selection import train_test_split

model = neighbors.NearestNeighbors(n_neighbors=8, algorithm='ball_tree')
model.fit(feature)
dist, idlist=model.kneighbors(feature)

In [ ]:
dist

In [ ]:
idlist

### Tesing Samples 

In [ ]:
def bookRecom(name):
    book_list=[]
    book_id = booksDataset_2[booksDataset_2['title']==name].index
    book_id = book_id[0]
    for new in idlist[book_id]:
        book_list.append(booksDataset_2.loc[new].title)
        book_list.append(booksDataset_2.loc[new].thumbnail)
    return book_list

bookRecommendation = bookRecom("Open City 6")
bookRecommendation

## Import Flask to running in HTML Page

In [ ]:
import requests
import sys
from flask import Flask, render_template, request
app = Flask(__name__, template_folder='templates')

In [ ]:
@app.route('/', methods=['GET', 'POST'])
def index():
    errors = []
    results = []
    if request.method == "POST":
        try:
            url = request.form['q']
            if url.strip()=="":
                    errors.append("Please enter a word or a book name.")
                    print(errors)
            else:
                for i in range(len(booksDataset)):
                    if (url.lower() in booksDataset['title'][i].lower()):
                        Book_name = bookRecom(booksDataset['title'][i])
                        break
                results.append(Book_name)
        except:
            errors.append(
                "The current word or book not matching, Please try another word or another book."
            )
            print(errors)
    return render_template('homePage.html', errors=errors, results=results)

In [ ]:
if __name__ == '__main__':
    app.run(debug=True, use_reloader=False)